In [8]:
import requests
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy import sparse
from bs4 import BeautifulSoup
import re
from transformers import DistilBertForSequenceClassification, DistilBertTokenizer
import torch
from transformers import pipeline
from util.config import config
from util.pyBM25 import BM25

In [51]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased-finetuned-sst-2-english')
classifier = pipeline(task='sentiment-analysis',model=model,tokenizer=tokenizer)
max_tokens = int(tokenizer.model_max_length)

In [9]:
stopwords=[]
with open('util/stopwords.txt') as f:
    stopwords.append(f.read().splitlines())

In [10]:
print(len(stopwords[0]))

431


In [3]:
bm25 = BM25(stopwords=stopwords)

NameError: name 'BM25' is not defined

In [ ]:
def query_Usearch(query, d_start, d_end, pages=1, pageSize=150):
    url = "https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/search/NewsSearchAPI"
    querystring = {"q":query,
                   "pageNumber":"1",
                   "pageSize":"150",
                   "autoCorrect":"false",
                   "fromPublishedDate":d_start,
                   "toPublishedDate":d_end}

    headers = {
        'x-rapidapi-host': "contextualwebsearch-websearch-v1.p.rapidapi.com",
        'x-rapidapi-key': "954c6fbd55msh55107080293cea6p148e40jsn1db2b31b95fd"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)
    
    

In [17]:
url = "https://contextualwebsearch-websearch-v1.p.rapidapi.com/api/search/NewsSearchAPI"

q = "PLUG"
d_from ="11/01/2021"
d_to = "null"

querystring = {"q":q,"pageNumber":"1","pageSize":"150","autoCorrect":"false","fromPublishedDate":d_from,"toPublishedDate":d_to}

headers = {
    'x-rapidapi-host': "contextualwebsearch-websearch-v1.p.rapidapi.com",
    'x-rapidapi-key': "954c6fbd55msh55107080293cea6p148e40jsn1db2b31b95fd"
    }

response = requests.request("GET", url, headers=headers, params=querystring)


In [18]:
response = response.json()

In [19]:
for i in response['value'][0]:
    print(i)

id
title
url
description
body
snippet
keywords
language
isSafe
datePublished
provider
image


In [20]:
corpus=[]
for i in response['value']:
    corpus.append(i['body'])

In [13]:
print(response['value'][0]['datePublished'])

2021-11-12T13:24:00


In [21]:
for i in response['value']:
    print(i['url'])

https://www.androidcentral.com/amazon-smart-plug-review
https://insideevs.com/news/547155/france-plugin-car-sales-october2021/
http://fool.com/investing/2021/11/11/why-plug-power-bloom-fuelcell-energy-just-popped/
https://mybigplunge.com/startups/zomato-pulls-the-plug-on-international-operations/
https://www.drive.com.au/news/porsche-911-hybrid-wont-be-a-plug-in-says-company-ceo-report/
https://www.nasdaq.com/articles/plug-crosses-above-average-analyst-target-2021-11-12
https://www.thisismoney.co.uk/money/markets/article-10192897/Footsie-giant-Johnson-Matthey-pulls-plug-electric-car-batteries.html
https://www.cbc.ca/news/canada/manitoba/springfield-icehawks-hockey-tournament-cancelled-covid-19-1.6245962?cmp=rss
https://stockcharts.com/articles/dont_ignore_this_chart/2021/11/plug-in-your-batteries-755.html
https://www.fonegiant.com/products/iphone-11-64gb-green-unlocked-excellent-condition
https://eurweb.com/2021/11/11/jeffcars-coms-review-2021-bmw-x5-xdrive45e-plug-in/
http://bnnbloomb

In [24]:
q = "Plug Power Inc"
bm25 = BM25()
bm25.fit(corpus)
result_vec = bm25.transform(q, corpus)
print(result_vec)

[1.52834383 0.57214766 1.61320328 0.2482312  1.56571111 5.94731821
 0.         0.51002331 1.68786242 0.44283014 2.10741846 0.41328758
 0.         1.44072037 1.47475431 3.73282215 0.21124267 0.
 0.51334896 0.54884515 0.         3.14755982 0.33132302 2.05499386
 2.79773621 0.55152728 1.54427807 0.32162401 0.         0.
 5.7444067  0.46901852 0.45509539 0.35007765 0.37263726 0.54020961
 0.32786252 0.41316584 2.61183031 1.3931969  0.55287433 1.92157792
 3.03400972 0.99937391 3.27967445 3.27967445 0.16926132 1.65236416
 0.44309356 0.53008327]


In [28]:
q = "Technology"
bm25 = BM25()
bm25.fit(corpus)
result_vec = bm25.transform(q, corpus)
print(result_vec)

[0.         0.         0.         0.         1.60615814 0.
 2.29717741 0.         2.6389479  0.         0.         0.
 0.         1.59355096 1.58522242 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 2.30536037 0.         0.         0.         0.         0.
 0.         2.09118818 1.97413543 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         1.9047118
 0.         0.        ]


In [27]:
print(response['value'][np.argmax(result_vec)]['url'])

https://www.nasdaq.com/articles/plug-crosses-above-average-analyst-target-2021-11-12


In [25]:
for i in range(0, len(response['value'])):
    print(result_vec[i],response['value'][i]['url'])
    

1.5283438269894027 https://www.androidcentral.com/amazon-smart-plug-review
0.5721476595861023 https://insideevs.com/news/547155/france-plugin-car-sales-october2021/
1.61320327859121 http://fool.com/investing/2021/11/11/why-plug-power-bloom-fuelcell-energy-just-popped/
0.2482312011274162 https://mybigplunge.com/startups/zomato-pulls-the-plug-on-international-operations/
1.5657111115187639 https://www.drive.com.au/news/porsche-911-hybrid-wont-be-a-plug-in-says-company-ceo-report/
5.947318207937107 https://www.nasdaq.com/articles/plug-crosses-above-average-analyst-target-2021-11-12
0.0 https://www.thisismoney.co.uk/money/markets/article-10192897/Footsie-giant-Johnson-Matthey-pulls-plug-electric-car-batteries.html
0.5100233070924018 https://www.cbc.ca/news/canada/manitoba/springfield-icehawks-hockey-tournament-cancelled-covid-19-1.6245962?cmp=rss
1.6878624232148256 https://stockcharts.com/articles/dont_ignore_this_chart/2021/11/plug-in-your-batteries-755.html
0.44283013552729716 https://ww

In [132]:
# from requests_html import AsyncHTMLSession
# session = AsyncHTMLSession()
# r = session.get(uurl)
# r = r.result()
# print(r.html.text)

In [71]:
def build_corpus_fromurl(url_list):
    corpus = []
    headers = {"User-Agent":"Mozilla/5.0"}
    for i in url_list:
        response = requests.get(url=i['url'],headers=headers)
        if response.status_code==200:
            soup = BeautifulSoup(response.content, 'html.parser')
            corpus.append(soup.get_text())
    return corpus

In [14]:
#corpus = build_corpus_fromurl(response['value'])

In [72]:
def get_pgraphs(soup):
    pgraphs=[]
    #Checking for 
    #Searching for html paragraphs
    if soup.find('p')!=None:
        freshsoup = BeautifulSoup(str(soup("p")),'html.parser')
        for p in freshsoup("p"):
            pgraphs.append(p.get_text())
    else:
        freshsoup = re.split('\n\n',soup.get_text())
        for x in range(0,len(freshsoup)):
            if len(freshsoup[x])>3:
                pgraphs.append(freshsoup[x])
    return pgraphs

In [80]:
def get_subdocs(pgraphs, max_tokens, min_sen=4):
    sub_docs=[]

    for x in range(0, len(pgraphs)):
        sen_cnt = len(re.split('\n|\. ',pgraphs[x]))
        tkns = int(len(tokenizer(pgraphs[x])['input_ids']))
        
        if tkns>=max_tokens:
                cut_point = pgraphs[x].rfind(".",0,int(len(pgraphs[x])/2))+1
                sen2 = pgraphs[x][cut_point:]
                sen = pgraphs[x][0:cut_point]
                # Recursively call get_subdocs by splitting the paragraph in half, if there are too many tokens
                get_subdocs(sen, max_tokens)
                get_subdocs(sen2, max_tokens)
                #sub_docs.append(sen)
                #sub_docs.append(sen2)
        else:
            sub_docs.append(pgraphs[x])
        

    
    return sub_docs

In [81]:
doc_cutoff = 0.25
subdoc_cutoff = 0

for i in response['value']:
    print(i['url'])
    headers = {"User-Agent":"Mozilla/5.0"}
    uspon = requests.get(url=i['url'],headers=headers)
    if uspon.status_code==200:
        soup = BeautifulSoup(uspon.content, 'html.parser')

        pgraphs = get_pgraphs(soup)
        sub_docs = get_subdocs(pgraphs,max_tokens)
        
        bm25.fit(sub_docs)
        result_vec = bm25.transform(q, sub_docs)
        
        sentiments = []
        for x in range(0,len(sub_docs)):
            if result_vec[x]>0:
                print(len(tokenizer(sub_docs[x])['input_ids']))
                s = classifier(sub_docs[x])
                scr = s[0]['score']
                if s[0]['label']=="NEGATIVE":
                    scr=scr*-1
                sentiments.append(scr)
    else:
        print(uspon.status_code)

https://www.fool.com/investing/2021/11/09/ballard-power-bloom-energy-nikola-stocks-drop/?source=iedfolrf0000001
96
85
65
https://timesofmalta.com/articles/view/shopify-plug-in-introduced-to-ecabs-fastdrop-platform.913614
78
67
https://arstechnica.com/?p=1811536
56
73
73
37
https://nesn.com/2021/11/tom-brady-responds-to-aaron-rodgers-question-with-shameless-plug/
24
https://www.refinery29.com/en-us/2021/11/10740548/maude-cone-butt-plug?utm_source=feed&utm_medium=rss
11
377
384
30
https://in.pcmag.com/smart-plugs/146046/kasa-smart-wi-fi-outdoor-plug-in-dimmer-kp-405
165
138
104
183
83
135
102
153
https://insideevs.com/news/546198/uk-plugin-car-sales-october2021/
53
29
66
35
27
https://trailer-wiring-diagram.com/7-core-trailer-wiring-diagram/narva-trailer-plug-wiring-diagram-7-core-trailer-wiring-diagram/
https://investorplace.com/2021/11/plug-powers-rally-is-just-beginning/
12
79
91
99
54
109
80
62
125
20
37
https://www.prweb.com/releases/autonomous_holds_7_day_launch_event_with_savings_

In [82]:
print(sentiments)

[-0.981147289276123, -0.9988498687744141, 0.9922612309455872, 0.9609275460243225]


In [184]:
q = "PLUG"
bm25 = BM25()
bm25.fit(corpus)
result_vec = bm25.transform(q, subs)
print(result_vec)
print("Length: ",len(result_vec))
print("Mean Score: ",np.mean(result_vec))
print("Median Score: ", np.median(result_vec))

[0.0433606  0.04105189 0.03070828 0.        ]
Length:  4
Mean Score:  0.028780191640045347
Median Score:  0.03588008440179806
